In [4]:
import pandas as pd

df= pd.read_json('../data_collect/website1/gen_data.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2427 entries, 0 to 2426
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2427 non-null   int64 
 1   name        2427 non-null   object
 2   text        2427 non-null   object
 3   accecpted   2427 non-null   int64 
 4   submission  2427 non-null   int64 
 5   topics      2427 non-null   object
 6   difficulty  2427 non-null   object
 7   link        2427 non-null   object
 8   website     2427 non-null   object
 9   upvotes     2427 non-null   int64 
 10  comments    2427 non-null   int64 
dtypes: int64(5), object(6)
memory usage: 227.5+ KB


In [ ]:
print(df.columns)
df= df[['id', 'name', 'text', 'topics', '']]

## Basic Preprocessing 

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from copy import deepcopy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from spellchecker import SpellChecker
import re
import string

stop_words = set(stopwords.words('english'))
math_pattern = r'\b[\d\+\-\*/=\(\)\s]+\b'
addit= ['given', 'follow', 'exampl', 'consist', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']
for nw in addit:
	stop_words.add(nw)

stemmer= PorterStemmer()

# Text Preprocessing
def Basic_preprocess(text):
	text = text.lower()
	text = text.translate(str.maketrans('', '', string.punctuation))
  
	# tokenizing the text
	tokens = word_tokenize(text)
	
	# initializing the spell checker
	spell = SpellChecker()
	inco_spell= spell.unknown(tokens);	final_tokens= ""

	# using spell checker to correct the spelling of the word
	for word in tokens:

		if(word in inco_spell):
			new_word= spell.correction(word)
		else:
			new_word= word

		if(new_word not in stop_words ):
			temp= re.sub(r'\d+', '', new_word)
			temp = re.sub(math_pattern, '', temp)

			if(temp!=''):
				new_word= stemmer.stem(word)
				final_tokens+= new_word + " "

	return final_tokens

questions= deepcopy(df)
for i, row in enumerate(questions['text']):
  questions.loc[i, 'text']= Basic_preprocess(row)

[nltk_data] Downloading package stopwords to /home/udayb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/udayb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 